<a href="https://colab.research.google.com/github/WillPinto1995/T2-IA/blob/main/TRAB2IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets scikit-learn telegram


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
from datasets import load_dataset

# Defina o caminho para o Dataset2
# Update this path to the actual location of your Dataset2 folder
dataset_path = '/content/drive/MyDrive/Dataset2'  # Example using Google Drive

# Carregar o dataset
def load_data_from_directory(directory):
    texts = []
    labels = []
    label_map = {'Basquete': 0, 'Futebol': 1, 'Vôlei': 2}  # Mapeamento das classes

    for label, label_id in label_map.items():
        class_folder = os.path.join(directory, label)
        # Check if the directory exists before trying to list its contents
        if os.path.exists(class_folder):
            for filename in os.listdir(class_folder):
                with open(os.path.join(class_folder, filename), 'r', encoding='utf-8') as file:
                    texts.append(file.read())
                    labels.append(label_id)
        else:
            print(f"Warning: Directory not found: {class_folder}")

    return texts, labels

# Carregar os dados de treino, validação e teste
train_texts, train_labels = load_data_from_directory(os.path.join(dataset_path, 'Train'))
val_texts, val_labels = load_data_from_directory(os.path.join(dataset_path, 'Val'))
test_texts, test_labels = load_data_from_directory(os.path.join(dataset_path, 'Test'))

# Exibir uma amostra
print(f'Exemplo de treino: {train_texts[0]}, Classe: {train_labels[0]}')

Exemplo de treino: Equipe
Ter no mínimo 8 jogadores aptos a jogar (5 em quadra e 3 no banco.) e no máximo 12
jogadores aptos a jogar.
Deve ter um técnico, e se o time preferir um assistente de técnico.
Deve também possuir um capitão do time.
Deve ter um uniforme com uma cor predominante e uma segunda cor.
•Início do jogo
– O Jogo começa com o lançamento da bola ao ar, pelo árbitro, entre dois jogadores
adversários no círculo central e esta só pode ser tocada quando atingir o ponto mais
alto.A equipe que não ganhou a posse de bola fica coma seta a seu favor.
•Duração do jogo
– Quatro períodos de 10minutos de tempo útil cada (Na NBA, são 12minutos), com um
intervalo de meio tempo entre o segundo e o terceiro período com uma duração de15
minutos, e com intervalos de dois minutos entre o primeiro e o segundo período e
entre o terceiro e o quarto período. O cronômetro só avança quando a bola se
encontra em jogo, isto é,sempre que o árbitro interrompe o jogo, o tempo é parado de
imediato.
•R

In [ ]:
from transformers import AutoTokenizer

# Usaremos o modelo 'bert-base-uncased' como exemplo
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_data(texts):
    return tokenizer(texts, padding=True, truncation=True, max_length=512)

# Tokenizando os textos de treino, validação e teste
train_encodings = tokenize_data(train_texts)
val_encodings = tokenize_data(val_texts)
test_encodings = tokenize_data(test_texts)



In [ ]:
from datasets import Dataset

# Criando os datasets para o treino, validação e teste
train_dataset = Dataset.from_dict({'input_ids': train_encodings['input_ids'], 'attention_mask': train_encodings['attention_mask'], 'labels': train_labels})
val_dataset = Dataset.from_dict({'input_ids': val_encodings['input_ids'], 'attention_mask': val_encodings['attention_mask'], 'labels': val_labels})
test_dataset = Dataset.from_dict({'input_ids': test_encodings['input_ids'], 'attention_mask': test_encodings['attention_mask'], 'labels': test_labels})


In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

# Configurações de treino
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

# Função para treinar um modelo
def train_model(model_name):
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
    )
    trainer.train()
    return trainer

# Treinando os dois modelos
trainer_bert = train_model("bert-base-uncased")
trainer_distilbert = train_model("distilbert-base-uncased")


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-16-48ccdc8fa8e9>:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

...

Epoch,Training Loss,Validation Loss
1,No log,0.958329
2,No log,1.133089
3,No log,1.033453


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-16-48ccdc8fa8e9>:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,No log,1.089052
2,No log,1.079707
3,No log,1.075835


In [ ]:
from sklearn.metrics import classification_report

# Função para avaliar o modelo
def evaluate_model(trainer, test_dataset):
    predictions = trainer.predict(test_dataset)
    preds = predictions.predictions.argmax(axis=-1)
    return classification_report(test_labels, preds)

# Avaliar os modelos
print("Avaliação do BERT:")
print(evaluate_model(trainer_bert, test_dataset))

print("Avaliação do DistilBERT:")
print(evaluate_model(trainer_distilbert, test_dataset))


Avaliação do BERT:


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.33      1.00      0.50         1
           2       0.00      0.00      0.00         1

    accuracy                           0.33         3
   macro avg       0.11      0.33      0.17         3
weighted avg       0.11      0.33      0.17         3

Avaliação do DistilBERT:


              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      1.00      1.00         1
           2       1.00      1.00      1.00         1

    accuracy                           1.00         3
   macro avg       1.00      1.00      1.00         3
weighted avg       1.00      1.00      1.00         3



In [ ]:
from sklearn.metrics import classification_report

# Fazer previsões para cada modelo
y_true = test_labels  # Verdadeiros rótulos das classes. Changed test_data['label'] to test_labels

# Assuming trainer_bert and trainer_distilbert from previous cell are your models
model1_trainer = trainer_bert  # Assign trainer_bert to model1_trainer
model2_trainer = trainer_distilbert  # Assign trainer_distilbert to model2_trainer

# Modelo 1
model1_predictions = model1_trainer.predict(test_dataset)
# Instead of using test_dataset.features['labels'], directly use the predicted labels
y_pred_model1 = [pred for pred in model1_predictions.predictions.argmax(axis=1)]
# Convert numerical predictions to label strings if necessary
# For example, if label 0 corresponds to "negative" and label 1 to "positive":
# y_pred_model1 = ["negative" if pred == 0 else "positive" for pred in y_pred_model1]

# Modelo 2
model2_predictions = model2_trainer.predict(test_dataset)
# Instead of using test_dataset.features['labels'], directly use the predicted labels
y_pred_model2 = [pred for pred in model2_predictions.predictions.argmax(axis=1)]
# Convert numerical predictions to label strings if necessary
# For example, if label 0 corresponds to "negative" and label 1 to "positive":
# y_pred_model2 = ["negative" if pred == 0 else "positive" for pred in y_pred_model2]

In [ ]:
# Classification report para o Modelo 1
print("Classification Report - Modelo 1")
# Assuming your labels are 0, 1, 2, you can create a list of target names:
target_names = ['label_0', 'label_1', 'label_2']  # Replace with your actual label names if different
report_model1 = classification_report(y_true, y_pred_model1, target_names=target_names)
print(report_model1)

# Classification report para o Modelo 2
print("Classification Report - Modelo 2")
report_model2 = classification_report(y_true, y_pred_model2, target_names=target_names)
print(report_model2)

Classification Report - Modelo 1
              precision    recall  f1-score   support

     label_0       0.00      0.00      0.00         1
     label_1       0.33      1.00      0.50         1
     label_2       0.00      0.00      0.00         1

    accuracy                           0.33         3
   macro avg       0.11      0.33      0.17         3
weighted avg       0.11      0.33      0.17         3

Classification Report - Modelo 2
              precision    recall  f1-score   support

     label_0       1.00      1.00      1.00         1
     label_1       1.00      1.00      1.00         1
     label_2       1.00      1.00      1.00         1

    accuracy                           1.00         3
   macro avg       1.00      1.00      1.00         3
weighted avg       1.00      1.00      1.00         3



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
model1_trainer.save_model("best_model")


In [ ]:
import importlib.metadata
print(importlib.metadata.version("python-telegram-bot"))


21.7


In [ ]:
# Certifique-se de instalar a biblioteca mais recente
!pip install --upgrade python-telegram-bot

# Verificar a versão instalada
import importlib.metadata
print(importlib.metadata.version("python-telegram-bot"))

# Importando os componentes corretamente com base na versão mais recente
from telegram.update import Update  # Agora Import Update de telegram.update
from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, filters, ContextTypes

# Inicializando o bot do Telegram
TOKEN = "7658169792:AAH0sFHzLBK_74MzTJyrl4mZSz5RaOyWmG4"

# Função para processar as mensagens
async def classify_text(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user_text = update.message.text
    inputs = tokenizer(user_text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    model = trainer_bert.model  # Use o modelo escolhido

    outputs = model(**inputs)
    predictions = outputs.logits.argmax(axis=-1).item()

    labels = {0: "Basquete", 1: "Futebol", 2: "Vôlei"}
    response = f"O texto foi classificado como: {labels[predictions]}"

    await update.message.reply_text(response)

# Criar a aplicação
app = ApplicationBuilder().token(TOKEN).build()

# Definir os handlers
app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, classify_text))

# Iniciar o bot
print("Bot iniciado. Pressione Ctrl+C para interromper.")
app.run_polling()


21.7


ModuleNotFoundError: No module named 'telegram.update'